In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt


In [2]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        #X_train, X_test, y_train, y_test = train_test_split(indep_X,dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test
    
def r2_prediction(regressor,X_test,y_test):
     y_pred = regressor.predict(X_test)
     from sklearn.metrics import r2_score
     r2=r2_score(y_test,y_pred)
     return r2
 
def Linear(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.linear_model import LinearRegression
        regressor = LinearRegression()
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2   
    
def svm_linear(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'linear')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
    
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVR
        regressor = SVR(kernel = 'rbf')
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def Decision(X_train,y_train,X_test):
        
        # Fitting K-NN to the Training setC
        from sklearn.tree import DecisionTreeRegressor
        regressor = DecisionTreeRegressor(random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2  
     

def random(X_train,y_train,X_test):       
        # Fitting K-NN to the Training set
        from sklearn.ensemble import RandomForestRegressor
        regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
        regressor.fit(X_train, y_train)
        r2=r2_prediction(regressor,X_test,y_test)
        return  r2 
    
def rfeFeature(indep_X, dep_Y, n):
    rfelist = []

    from sklearn.linear_model import LinearRegression
    from sklearn.svm import SVR
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor

    lin = LinearRegression()
    SVRl = SVR(kernel='linear')
    dec = DecisionTreeRegressor(random_state=0)
    rf = RandomForestRegressor(n_estimators=10, random_state=0)

    rfemodellist = [
        ('Linear', lin),
        ('SVM', SVRl),
        ('DecisionTree', dec),
        ('RandomForest', rf)
    ]

    for name, model in rfemodellist:
        rfe = RFE(model, n)
        rfe.fit(indep_X, dep_Y)

        selected_columns = indep_X.columns[rfe.support_]
        X_rfe = indep_X[selected_columns]

        rfelist.append((name, rfe, X_rfe))

    return rfelist

    
def rfe_regression(acclog,accsvml,accdes,accrf): 
    
    rfedataframe=pd.DataFrame(index=['Linear','SVC','Random','DecisionTree'],columns=['Linear','SVMl',
                                                                                        'Decision','Random'])

    for number,idex in enumerate(rfedataframe.index):
        
        rfedataframe['Linear'][idex]=acclog[number]       
        rfedataframe['SVMl'][idex]=accsvml[number]
        rfedataframe['Decision'][idex]=accdes[number]
        rfedataframe['Random'][idex]=accrf[number]
    return rfedataframe

In [3]:

dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', 1)
dep_Y=df2['classification_yes']


In [4]:
rfelist=rfeFeature(indep_X,dep_Y,3) 

C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklear

In [5]:
rfelist = rfeFeature(indep_X, dep_Y, 3)

acclin = []
accsvml = []
accdes = []
accrf = []

for name, rfe_model, X_rfe in rfelist:
    
    print(f"\nRFE Base Model: {name}")
    print("Selected Columns:", list(X_rfe.columns))

    X_train, X_test, y_train, y_test = split_scalar(X_rfe, dep_Y)

    acclin.append(Linear(X_train, y_train, X_test))
    accsvml.append(svm_linear(X_train, y_train, X_test))
    accdes.append(Decision(X_train, y_train, X_test))
    accrf.append(random(X_train, y_train, X_test))


C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklear


RFE Base Model: Linear
Selected Columns: ['sg_c', 'sg_d', 'sg_e']

RFE Base Model: SVM
Selected Columns: ['sg_c', 'sg_d', 'sg_e']

RFE Base Model: DecisionTree
Selected Columns: ['hrmo', 'sg_c', 'sg_d']

RFE Base Model: RandomForest
Selected Columns: ['al', 'hrmo', 'sg_d']


C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):


In [6]:
    
result=rfe_regression(acclin,accsvml,accdes,accrf)

In [7]:
result

,Linear,SVMl,Decision,Random
Linear,0.441961,0.262153,0.441961,0.441816
SVC,0.441961,0.262153,0.441961,0.441816
Random,0.664893,0.609652,0.965961,0.916304
DecisionTree,0.676174,0.670691,0.933504,0.887256


In [9]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

def print_rfe_selected_columns(indep_X, dep_Y, n_features=3):
    
    models = {
        "Linear": LinearRegression(),
        "SVM_linear": SVR(kernel='linear'),
        "DecisionTree": DecisionTreeRegressor(random_state=0),
        "RandomForest": RandomForestRegressor(n_estimators=10, random_state=0)
    }
    
    for name, model in models.items():
        rfe = RFE(model, n_features)
        rfe.fit(indep_X, dep_Y)
        
        selected_cols = indep_X.columns[rfe.support_]
        
        print(f"\nRFE Base Model: {name}")
        print("Selected Columns:")
        print(list(selected_cols))


In [10]:
print_rfe_selected_columns(indep_X, dep_Y, n_features=3)



RFE Base Model: Linear
Selected Columns:
['sg_c', 'sg_d', 'sg_e']

RFE Base Model: SVM_linear
Selected Columns:
['sg_c', 'sg_d', 'sg_e']

RFE Base Model: DecisionTree
Selected Columns:
['hrmo', 'sg_c', 'sg_d']


C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklear


RFE Base Model: RandomForest
Selected Columns:
['al', 'hrmo', 'sg_d']


C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Mohamed Imran\New folder\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
